# 4.2 Evaluar cuantitativamente y cualitativamente algoritmos para detectar cambio semántico

Se evaluaran Orthogonal Procrustes ,Second-Order Similarity Y NN (Nearest Neighbors) para detectar ara detectar cambios en el significado de las palabras en los títulos de IL a lo largo de los años 2010 - 2023.

Ver si es así
* Si buscas estabilidad y comparabilidad: Orthogonal Procrustes es la mejor opción.
* Si quieres capturar cambios más sutiles: Second-Order Similarity es más adecuado.
* Si te interesa la interpretación basada en contexto: Compass es una buena elección.
* Si prefieres una detección simple y efectiva: NN (Nearest Neighbors) es fácil de aplicar.

📌 Consideraciones antes de aplicar los métodos
* 1️⃣ Tamaño del corpus: Si el período 2020-2024 tiene menos datos, es posible que el análisis de cambio semántico sea más ruidoso. Puedes compensarlo con técnicas de normalización o usando menos palabras con baja frecuencia.
* 2️⃣ Alineación temporal: Debes alinear los embeddings de manera secuencial (2010-2015 → 2015-2020 → 2020-2024) usando Orthogonal Procrustes o Compass.
* 3️⃣ Comparación estable: Para evitar sesgos, podrías analizar cambios entre períodos completos (2010-2020) y luego comparar con el período incompleto (2020-2024).
* 4️⃣ Intersección de vocabulario: Eliminar palabras con muy baja frecuencia en el período 2020-2024 para evitar palabras con embeddings inestables.
*  Comparar con períodos combinados: Puedes comparar 2010-2020 vs. 2020-2024 para ver si los cambios en el último período son significativos.

### Papers recientes sobre cambio semántico
* Hamilton et al. (2016) - "Diachronic Word Embeddings Reveal Statistical Laws of Semantic Change"
    * https://aclanthology.org/P16-1141/
    * Introduce Orthogonal Procrustes para alinear embeddings.

* Gonen et al. (2020) - "Simple, Interpretable and Stable Method for Detecting Words with Usage Change across Corpora"
    * https://aclanthology.org/2020.acl-main.51/
    * Introduce el método basado en vecinos más cercanos (NN).

* Montariol et al. (2021) - "Scalable and Interpretable Semantic Change Detection"
    * https://aclanthology.org/2021.naacl-main.369/
    * Combina varios métodos para mejorar la detección de cambio semántico
      
      
* Dritsa1 et al. (2022) - "A Greek Parliament Proceedings Dataset for Computational Linguistics and Political Analysis"
    * El paper analiza el cambio semántico en discursos parlamentarios y compara cuatro enfoques principales para detectar cambios en el significado de palabras a lo largo del tiempo.


In [1]:
# Importar librerias
import os
import pandas as pd
import numpy as np
import pickle
import sweetviz as sv

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from matplotlib import colormaps


from tqdm import tqdm
tqdm.pandas()

In [2]:
# Configurar path
os.chdir('C://iamas_datos2024/proyectos_parlamentarios_2025/')
pd.set_option('display.max_colwidth', None)

In [3]:
# Abrir
df = pickle.load(open('filtro_texto_df_ley_1023.pkl', 'rb'))

In [4]:
df.head(2)

,Proyecto.ID,Título,Título normalizado,Proyecto_girado_a_comisiones_SALUD,Proyecto_SALUD,Resultado,Max_Orden,Tiene_antecedente_por_titulo_proy,Periodo,Tokens,Publicación.Fecha,Año,Duración_dias_prep
0,HCDN272363,DECLARESE EL 2024 COMO AÑO DEL 140 ANIVERSARIO DE LA PROMULGACION DE LA LEY 1420 DE EDUCACION COMUN.,declarese aniversario promulgacion educacion comun,GIRADO A OTRAS COMISIONES,0.0,NO TUVO TRATAMIENTO POSTERIOR NI DICTAMEN,2,False,NaN,"[declarese, aniversario, promulgacion, educacion, comun]",2023-12-29,2023,-1.0
1,HCDN272359,"FINANCIAMIENTO DE LOS PARTIDOS POLITICOS - LEY 26215 - Y CODIGO ELECTORAL NACIONAL - LEY 19945 -. MODIFICACIONES SOBRE CONTRATACION DE PUBLICIDAD Y BOLETA UNICA, RESPECTIVAMENTE.",financiamiento partidos politicos electoral nacional modificaciones contratacion publicidad boleta unica respectivamente,GIRADO A OTRAS COMISIONES,0.0,NO TUVO TRATAMIENTO POSTERIOR NI DICTAMEN,1,False,NaN,"[financiamiento, partidos, politicos, electoral, nacional, modificaciones, contratacion, publicidad, boleta, unica, respectivamente]",2023-12-28,2023,-1.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31334 entries, 0 to 31333
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Proyecto.ID                         31334 non-null  category      
 1   Título                              31334 non-null  category      
 2   Título normalizado                  31334 non-null  object        
 3   Proyecto_girado_a_comisiones_SALUD  31334 non-null  category      
 4   Proyecto_SALUD                      31334 non-null  float64       
 5   Resultado                           31334 non-null  category      
 6   Max_Orden                           31334 non-null  int64         
 7   Tiene_antecedente_por_titulo_proy   31334 non-null  bool          
 8   Periodo                             29781 non-null  float64       
 9   Tokens                              31334 non-null  object        
 10  Publicación.Fecha     

In [7]:
df = df[df['Proyecto_SALUD']==1]
print("Conj. de datos de proyecto_SALUD con Nan en Período:",df.shape)
df = df[~df['Periodo'].isna()]
print("Conj. de datos de proyecto_SALUD sin Nan en Período:",df.shape)

Conj. de datos de proyecto_SALUD con Nan en Período: (2692, 13)
Conj. de datos de proyecto_SALUD sin Nan en Período: (2562, 13)


###  Cargar Corpus para los tres períodos

In [8]:
df['Periodo_5anios'] = 2010
df.loc[(df['Año']>=2015) & (df['Año']<2020), 'Periodo_5anios'] =  2015
df.loc[(df['Año']>=2020) & (df['Año']<2024), 'Periodo_5anios'] =  2020

In [9]:
#concat sentences, each last sentence for each speech did not have dot so add one.
PER5anios_df = df.groupby('Periodo_5anios')['Título normalizado'].progress_apply('.'.join).reset_index() 
PER5anios_df['corpus']= PER5anios_df['Título normalizado'].progress_apply(lambda x: [sent.split(' ') for sent in x.split('.')])
PER5anios_df['corpus'] = PER5anios_df['corpus'].progress_apply(lambda x: [token for token in x if token!='' and token!=' '])


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2996.64it/s]


In [ ]:
# Guardar el archivo binario proyecto
with open('PER5anios_2025df.pkl', 'wb') as file:
    pickle.dump(PER5anios_df,file)

In [10]:
PER5anios_df['corpus'][0][0:3]

[['acciones',
  'instituyase',
  'julio',
  'homenaje',
  'nacimiento',
  'doctor',
  'rene',
  'geronimo',
  'favaloro'],
 ['cobertura', 'universal', 'salud', 'instituye', 'diciembre'],
 ['historias', 'clinicas', 'regimen']]

## 1 -  Orthogonal Procrustes (Hamilton)

In [11]:
from gensim.models import Word2Vec
import itertools
from collections import defaultdict
import re
import random as rn
import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
def compute_cosine_similarity(model1,model2,word):
    vector1 = model1.wv[word].reshape(1,-1)
    vector2 = model2.wv[word].reshape(1,-1)
    return(cosine_similarity(X=vector1, Y=vector2)[0][0])

def step_one_pairs(list_of_items):
    return [(list_of_items[i],list_of_items[i+1]) for i in range(len(list_of_items)-1)]

tqdm.pandas()

In [13]:
# Configuración
models_dir =  './archivos_salidas/modelos/hamilton_estabilidad/'
run=0
iterations=10
diter=5
siter=5
vector_size=200 #


In [14]:
PER5anios_df.columns

Index(['Periodo_5anios', 'Título normalizado', 'corpus'], dtype='object')

In [15]:
if not os.path.exists(models_dir):
    print('Creating models directory...')
    os.makedirs(models_dir)
    
PER5anios_df.sort_values(by='Periodo_5anios')
anios5_list = sorted(PER5anios_df['Periodo_5anios'].to_list())
anios5_pairs = step_one_pairs(anios5_list)
anios5_pairs


[(2010, 2015), (2015, 2020)]

In [ ]:
# Función para alinear embeddings
def align_embeddings(base_model, target_model, shifts_PERanios5_list, i,periodo):
    base_words = list(base_model.wv.index_to_key)
    target_words = list(target_model.wv.index_to_key)
    common_words = list(set(base_words) & set(target_words))
    print('Common vocab length... ', str(len(common_words)))
    print('Computing word similarity between ....')

    for word in tqdm(common_words):
        #union of neighbors in three points in time
        neighbors_t1 = [w for w,s in base_model.wv.most_similar(positive=[word], topn=25)]
        neighbors_t2 = [w for w,s in target_model.wv.most_similar(positive=[word], topn=25)]
        neighbors_union = [n for n in list(set(neighbors_t1+neighbors_t2)) if n in common_words]
        
        # similarity vector for time point 1 (t1)
        # each element is the cosine similarity of topic vector in t1 and each neighbor from neighbors_union
        similarity_vector_t1 = []
        word_vector_t1 = base_model.wv[word]
        
        for nn in neighbors_union:
            neighbor_vector = base_model.wv[nn]
            similarity_vector_t1.append(cosine_similarity(X=word_vector_t1.reshape(1,-1), Y=neighbor_vector.reshape(1,-1))[0][0])

        # similarity vector for time point 2 (t2)
        # each element is the cosine similarity of topic vector in t2 and each neighbor from neighbors_union
        similarity_vector_t2 = []
        word_vector_t2 = target_model.wv[word]
        
        for nn in neighbors_union:
            neighbor_vector = target_model.wv[nn]
            similarity_vector_t2.append(cosine_similarity(X=word_vector_t2.reshape(1,-1), Y=neighbor_vector.reshape(1,-1))[0][0])

      # final cosine between cosines/ similarity vectors for t1 and t2
        result1 = cosine_similarity(np.array([similarity_vector_t1]),
                                   np.array([similarity_vector_t2])
                                  )[0][0]
  
        shifts_PERanios5_list.append([i, periodo, word, result1, len(common_words), neighbors_t1, neighbors_t2, 
                                      len(neighbors_union)])
  
        
     
    return shifts_PERanios5_list

# Función para alinear múltiples períodos secuencialmente
def align_multiple_periods(models,shifts_PERanios5_list, i):
    
    for j in range(1, len(models)):
        periodo = anios5_pairs[j - 1]
        shifts_PERanios5_list = align_embeddings(models[j - 1], models[j], shifts_PERanios5_list, i, periodo)
       
    return shifts_PERanios5_list

In [ ]:
shifts_PERanios5_list = []
for i in range(iterations):
    
    np.random.seed(i)
    rn.seed(i)
    my_seed=i
        
    print('********************************************************')
    print('Repeat No ', str(i))
    
    print(datetime.datetime.now())
    
    print('Training models for each anios5...')

    for anios5, texts in tqdm(zip(PER5anios_df['Periodo_5anios'], PER5anios_df['Título normalizado'])):
        print(anios5)
        model = Word2Vec(sentences=texts, vector_size=vector_size, window=5, min_count=20, workers=1, seed=my_seed)
        model.save(models_dir+str(anios5)+'_'+str(i)+ ".mdl")

    print(datetime.datetime.now())

    modelos = []
    for anios in anios5_list:
        m1 = Word2Vec.load(models_dir+str(anios)+'_'+str(i)+ ".mdl")
        modelos.append(m1)
    
    # Alinear embeddings secuencialmente
    shifts_PERanios5_list = align_multiple_periods(modelos,shifts_PERanios5_list,i) 


    
shifts_PER5anios_df = pd.DataFrame(
    data=shifts_PERanios5_list, columns=['iteracion', 'anios5', 'palabra', 'similitud_semantica', 'vocabulario_comun', 'top25vecinos_1er_anios5', 'top25vecinos_2do_anios5', 'union_de_vecinos'])


shifts_PER5anios_df.describe()

shifts_PER5anios_df = shifts_PER5anios_df.sort_values('similitud_semantica', ascending=False).reset_index(drop=True)
print(shifts_PER5anios_df.head(5))
print('------------')
print(shifts_PER5anios_df.tail(5))

shifts_PER5anios_df.to_csv('./archivos_salidas/corrida_hamilton_estable'+str(run)+'_iteracion'+str(
    iterations)+'_tamanio'+str(vector_size)+'_filas'+str(no_rows)+'.csv', index=False)

topn_dict = {}
X = []
Y = []

k=[5,10,20,30,50,100,200,300,500,600,700,800,1000] #

for n in k:
    
    for iteration in range(iterations):
        subdf = shifts_PER5anios_df.loc[(shifts_PER5anios_df['iteracion']==iteration)]
        subdf = subdf.sort_values('similitud_semantica', ascending=False).reset_index(drop=True)
        topn_dict[iteration] = subdf.head(n)['palabra'].to_list()
    
    topn_list_of_lists = [val for key, val in topn_dict.items()]

    intersection = len(set(topn_list_of_lists[0]).intersection(*topn_list_of_lists))

    Y.append(intersection/n)
    X.append(n)
    
# print(X,Y)

fig = plt.figure(figsize=(15, 8))

fig.set_size_inches(20, 10)
plt.scatter(X,Y)
plt.plot(X,Y)
plt.gca().tick_params(axis='both', which='major', labelsize=15)
plt.ylim(0,1.)
plt.xlabel('k', fontsize=18)
plt.ylabel('Intersection@k', fontsize=18)
plt.title('Estabilidad por Hamilton', fontsize=20)

plt.savefig('./archivos_salidas/corrida_hamilton_estable'+str(run)+'_iteracion'+str(
    iterations)+'_tamanio'+str(vector_size)+'_filas'+str(no_rows)+'.png', dpi=200,  bbox_inches='tight')